In [11]:
import json

input_file = r"raw_dataset/train.jsonl"  # Update this if needed
output_file = r"processed/unsupervised_structured_train.jsonl"

In [ ]:
def extract_sections(data):
    title = data.get("title", "").strip()
    abstract = data.get("abstract", "").strip()
    full_text = data.get("full_text", {})
    section_names = full_text.get("section_name", [])
    paragraphs = full_text.get("paragraphs", [])

    introduction, conclusion, full_body = [], [], []

    for sec_name, paras in zip(section_names, paragraphs):
        if not sec_name or not paras or not isinstance(paras, list):
            continue  # Skip missing or invalid sections

        sec_name_lower = sec_name.lower()
        clean_paras = "\n".join(p.strip() for p in paras if isinstance(p, str) and p.strip()).strip()

        if not clean_paras:
            continue

        if "introduction" in sec_name_lower:
            introduction.append(clean_paras)
        elif "conclusion" in sec_name_lower:
            conclusion.append(clean_paras)
        else:
            full_body.append(f"### {sec_name}\n{clean_paras}")


    return {
        "title": title,
        "abstract": abstract,
        "introduction": "\n\n".join(introduction).strip(),
        "full_text": "\n\n".join(full_body).strip(),
        "conclusion": "\n\n".join(conclusion).strip()
    }

In [15]:
def main():
    count = 0
    with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
        for line in infile:
            data = json.loads(line)
            structured = extract_sections(data)

            if any(structured.values()):  # At least one field must be non-empty
                json.dump(structured, outfile)
                outfile.write("\n")
                count += 1

    print(f"✅ Done. {count} entries written to {output_file}")

if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'lower'